# Testing Regression vs. Galerkin Projection

In [1]:
# Imports
import numpy as np
import math
from __future__ import print_function
 

import PyUQTk.pce as uqtkpce
import PyUQTk.PyPCE.pce_tools as pce_tools
from PyUQTk.utils.func import *

import sys
sys.path.append('../pyuqtkarray/')
import PyUQTk.uqtkarray as uqtkarray

import pandas as pd

PyMC is required for some of the MCMC postprocessing codes.
Will proceed without, but some convergence tests will not be available.


## Inputs

In [2]:
nord = 2            # Order of the PCE
pc_type = "HG"      # Polynomial type
pc_alpha = 0.0      # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
pc_beta = 1.0       # Free parameter > -1 for Gamma-Laguerre and Beta-Jacobi PCs
param = nord+1      # Number of quadrature points per dimension for full quadrature or level for sparse quadrature
nSam = 100000       # Number of random samples
ndim = 2            # Number of dimensions

Define the polynomial model to matching the chosen PC type.

In [3]:
coef=np.ones(param)
if (ndim==1):
    if (pc_type=='LU'):
        poly = np.polynomial.legendre.Legendre(coef, domain=None, window=None)
    if (pc_type=='HG'):
        poly=np.polynomial.hermite_e.HermiteE(coef, domain=None, window=None)
    if (pc_type=='LG'):
        poly=np.polynomial.laguerre.Laguerre(coef, domain=None, window=None)
# 'JB', 'CC', 'CCO', 'NC', 'NCO', 'GP3', 'pdf'
    poly

## Quadrature

In [4]:
pc_model = uqtkpce.PCSet("NISPnoq", nord,ndim,pc_type, pc_alpha,pc_beta)
pc_model.SetQuadRule(pc_type, 'full', param)
qdpts, totquat= pce_tools.UQTkGetQuadPoints(pc_model)

In [5]:
pd.DataFrame(qdpts)

,0,1
0,-1.732051e+00,-1.732051e+00
1,-2.069764e-16,-1.732051e+00
2,1.732051e+00,-1.732051e+00
3,-1.732051e+00,-2.069764e-16
4,-2.069764e-16,-2.069764e-16
5,1.732051e+00,-2.069764e-16
6,-1.732051e+00,1.732051e+00
7,-2.069764e-16,1.732051e+00
8,1.732051e+00,1.732051e+00


In [6]:
f_evals=np.zeros(totquat)
q=[]
for i in range(totquat):
    #f_evals[i]=poly(qdpts[i])
    f_evals[i]=np.polynomial.hermite_e.hermeval2d(qdpts[i][0], qdpts[i][1], coef)
#pd.DataFrame(data=np.transpose([q, f_evals]), columns=["qdpts", "f_evals"])

## Finding Coefficients

In [7]:
c_k=pce_tools.UQTkRegression(pc_model, f_evals)

In [8]:
#psi_uqtk = uqtkarray.dblArray2D()
#pc_model.GetPsi(psi_uqtk)
#psi_np = np.zeros( (totquat, pc_model.GetNumberPCTerms()) )
#psi_np = uqtkarray.uqtk2numpy(psi_uqtk)
#c_k, resids, rank, s = np.linalg.lstsq(psi_np,f_evals,rcond=None)

In [9]:
c_k2 = pce_tools.UQTkGalerkinProjection(pc_model,f_evals)

In [10]:
pd.DataFrame(data=np.transpose([c_k, c_k2]), columns=['Regression', "Galerkin"])

,Regression,Galerkin
0,1.000000e+00,1.000000e+00
1,1.000000e+00,1.000000e+00
2,1.127942e-16,-6.938894e-16
3,1.000000e+00,1.000000e+00
4,2.926378e-16,-6.106227e-16
5,8.208866e-16,1.942890e-16


## Error

In [11]:
germ_samples=np.random.normal(0,1, (nSam,ndim))
pce_evals=pce_tools.UQTkEvaluatePCE(pc_model,c_k,germ_samples)

f_actual=np.zeros(nSam)
for i in range(nSam):
    f_actual[i]=poly(germ_samples[i])
    
MSE = np.square(np.subtract(f_actual,pce_evals)).mean()
RMSE=math.sqrt(MSE)

print("The RMSE is")
RMSE

NameError: name 'poly' is not defined